In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import pandas as pd
from DateProvider import DateProvider
from datetime import datetime
from ColumnsAdder import ColumnsAdder
from MedianOfFreeBedsByKreisTableFactory import MedianOfFreeBedsByKreisTableFactory
from TimeseriesReader import readTimeseries
from IntensivstationenHtmlFileUpdater import saveLastUpdatedIntensivstationen, saveKreisOptions
from KreiseReader import readKreise
from KreisOptionsProvider import getKreisOptionsAndAlleLandkreise
from IntensiveCareBedsPersister import getAndPersistIntensiveCareBeds4AlleKreise
from Chart import createMedianOfFreeBedsByKreisChart


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', 'raise')


In [3]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

04.02.2023, 04:14:28 Uhr


In [4]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = True # dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

          lastUpdated: 2023-02-02 11:49:00
lastUpdatedDataSource: 2023-02-03 11:49:00
needsUpdate: True


In [5]:
saveLastUpdatedIntensivstationen(
    dateProvider.getLastUpdatedDataSource(),
    toHtmlFile = "../../docs/intensivstationen.html")

In [6]:
timeSeries = readTimeseries(download = needsUpdate)
timeSeries

,date,bundesland,gemeindeschluessel,betten_frei,betten_belegt,Kreis,Einwohnerzahl,Bundesland
0,2020-04-24,01,01001,40,38,"Flensburg, Stadt",89934.0,Schleswig-Holstein
267,2020-04-24,09,09471,9,9,Bamberg,147497.0,Bayern
266,2020-04-24,09,09464,17,23,Hof,45173.0,Bayern
265,2020-04-24,09,09463,9,25,Coburg,40842.0,Bayern
264,2020-04-24,09,09462,12,51,Bayreuth,74048.0,Bayern
...,...,...,...,...,...,...,...,...
401873,2023-02-03,06,06437,5,139,Odenwaldkreis,96754.0,Hessen
401872,2023-02-03,06,06436,0,20,Main-Taunus-Kreis,239264.0,Hessen
401871,2023-02-03,06,06435,4,71,Main-Kinzig-Kreis,421689.0,Hessen
401869,2023-02-03,06,06433,6,17,Groß-Gerau,275807.0,Hessen


In [7]:
kreisValues = sorted(timeSeries['Kreis'].drop_duplicates().values)
saveKreisOptions(
    kreisOptions = getKreisOptionsAndAlleLandkreise(kreisValues),
    toHtmlFile = "../../docs/intensivstationen.html")

In [8]:
getAndPersistIntensiveCareBeds4AlleKreise(
    timeSeries,
    intensivstationenDataDir = '../../docs/data/intensivstationen',
    kreisValues = kreisValues)


None

'Ahrweiler'

'Aichach-Friedberg'

'Alb-Donau-Kreis'

'Altenburger Land'

...

'Wuppertal, Stadt'

'Würzburg'

'Zollernalbkreis'

'Zweibrücken, kreisfreie Stadt'

'Zwickau'

In [9]:
medianOfFreeBedsByKreisTable = MedianOfFreeBedsByKreisTableFactory(timeSeries).createMedianOfFreeBedsByKreisTable('Kreis')
medianOfFreeBedsByKreisTable.reset_index().to_json(
    '../../docs/data/intensivstationen/medianOfFreeBedsByKreisTable.json',
    orient = "records")

In [10]:
def getChartTitle(dateStart, dateEnd):
    def formatDate(date):
        return date.strftime("%d.%m.%Y")

    return f"Median freier Intensivbetten im Zeitraum {formatDate(dateStart)} bis {formatDate(dateEnd)}"

In [11]:
import os

createMedianOfFreeBedsByKreisChart(
    timeSeries,
    chartTitle = getChartTitle(
        dateStart=timeSeries['date'].min(),
        dateEnd=timeSeries['date'].max()),
    accessToken = os.environ['DATAWRAPPER_API_TOKEN'])

publishChart: {'data': {'publicId': 'dYmYb', 'language': 'de-DE', 'theme': 'datawrapper', 'id': 'dYmYb', 'type': 'd3-maps-choropleth', 'title': 'Median freier Intensivbetten im Zeitraum 24.04.2020 bis 03.02.2023', 'lastEditStep': 5, 'publishedAt': '2023-02-04T04:15:02.672Z', 'publicUrl': 'https://datawrapper.dwcdn.net/dYmYb/339/', 'publicVersion': 339, 'deleted': False, 'deletedAt': None, 'forkable': True, 'isFork': False, 'metadata': {'data': {'changes': [], 'transpose': False, 'vertical-header': True, 'horizontal-header': True, 'column-format': {'ags': {'type': 'text'}, 'Kreis': {'type': 'text'}, 'Einwohnerzahl': {'type': 'number', 'number-append': '', 'number-format': 'auto', 'number-divisor': 0, 'number-prepend': ''}, 'gemeindeschluessel': {'type': 'text'}, 'median_free_beds_in_percent': {'type': 'number'}}, 'upload-method': 'upload'}, 'describe': {'source-name': 'Landkreis-Daten', 'source-url': 'https://www.intensivregister.de/#/aktuelle-lage/downloads', 'intro': '', 'byline': '',